In [20]:
from db import DBhelper
from basic import timing, logging_channels
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
import requests
import urllib.request, urllib.error
import xlwt
import re
from random import shuffle
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib

def toemail(title,dict_):
    content = MIMEMultipart()  #建立MIMEMultipart物件
    content["subject"] = f"{title}"  #郵件標題
    content["from"] = "yu.jiang@avividai.com"  #寄件者
    content["to"] = "yu.jiang@avividai.com" #收件者
    content.attach(MIMEText(dict_))  #郵件內容
    with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:  # 設定SMTP伺服器
        try:
            smtp.ehlo()  # 驗證SMTP伺服器
            smtp.starttls()  # 建立加密傳輸
            smtp.login("yu.jiang@avividai.com", "mfitqgprhdpanvll")  # 登入寄件者gmail
            smtp.send_message(content)  # 寄送郵件
            print("Complete!")
        except Exception as e:
            print("Error message: ", e)
    return
def fetch_title_url():
    query = f"""
            SELECT

            web_id,product_id,title, url 
            FROM
                item_list
            
            

            """
    print(query)
    data = DBhelper('rhea1-db0').ExecuteSelect(query)
    df_article = pd.DataFrame(data, columns=['web_id', 'product_id', 'title', 'url'])
    return df_article


def fetch_rule(web_id):
    query = f"""
            SELECT

            title
            FROM
                web_id_url_encoder_rule

            WHERE
                web_id ="{web_id}"

            """
    print(f"web_id ={web_id}")
    title_rule = DBhelper('jupiter_new').ExecuteSelect(query)
    # df_article = pd.DataFrame(data, columns=['web_id','product_id','title', 'url'])
    return title_rule


def fetch_soup(url):
    try:
        html = requests.get(url, headers={
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:20.0) Gecko/20100101 Firefox/20.0'}, timeout=3)
        html.encoding = 'utf-8'
        soup = BeautifulSoup(html.text, 'lxml')

    except:
        soup = BeautifulSoup('0')
    return soup

def save2xlsx(data_wrong_id,path,web_id):
    data_wrong_id_df = pd.DataFrame.from_dict(data_wrong_id,"index")
    data_wrong_id_df.to_excel(f"""title_report/{path}/{web_id}_report.xlsx""",index=False)
    return

def similarity(title,title_online):
    if len(title)>len(title_online):
        long_title=title
        short_title=title_online
    else:
        long_title=title_online
        short_title=title
    if len(short_title)==0:
        return 0
    C_ratio=len([ i for i in short_title if i in [i for i in long_title]])/len(short_title)
    
    return C_ratio





if __name__ == '__main__':
#def title_wrong_report():
    data_count,j,k={},0,0
    data = fetch_title_url()
    web_id_list = list(set(data['web_id']))
    no_ruel_dict={}
    norule=''
    #report=pd.read_excel('title_report.xlsx')
    #web_id_d=list(report[report['data_correct']<0.8].web_id)
    #web_id_d=[i for i in web_id_list if i not in web_id_c]
    shuffle(web_id_list)
    for web_id in tqdm(web_id_list):
        error_count,data_wrong_count,url_wrong_count,data_c,no_rule = 0,0,0,0,0
        data_wrong_id,url_wrong_id,error_id={},{},{}
        o,t,f=0,0,0
        rule = fetch_rule(web_id)
        
        if rule:  ###判斷是否存在規則
            rule = rule[0][0]
        else:
            print(f'{web_id} no rule in table')
            data_count[j] = {'web_id': web_id, 'data_correct': data_c, 'data_wrong_count': data_wrong_count,
                             'error_count': error_count, 'url_wrong_count': url_wrong_count,'no_rule': 1}
            j += 1
            
            no_ruel_dict[k] ={'web_id':web_id,'wrong_type':4}
            
            k +=1
            norule+=f'web_id={web_id}\n'
            continue
        data_ = data.loc[data['web_id'].isin([web_id])]
        data_.sample(frac=1)  ###亂續
        for url, title, product_id in zip(data_['url'][:100], data_['title'][:100], data_['product_id'][:100]):
            soup = fetch_soup(url)
            try:
                content = soup.find('meta', attrs=eval(str(rule)))
            except:
                content=None

            if not content:
                try:
                    content = eval(rule) if str(eval(rule)).replace(' ','')!=rule else None 
                except:
                    content = None
                    
            if content:  ##判斷網址是否失效
                try:
                    title_online = content.attrs['content']
                except:
                    title_online = content
                
                C_ratio = similarity(title,title_online)

                if C_ratio<0.4:  ##判斷是否title是否相同
                    if re.search(url,str(soup)):
                        data_wrong_count+=1
                        data_wrong_id[o]={'web_id':web_id,'wrong_type':1,'title':title,'title_online':title_online,'url':url}
                        o=o+1
                    else:
                        url_wrong_count+=1
                        url_wrong_id[t]={'web_id':web_id,'wrong_type':2,'title':title,'title_online':title_online,'url':url}
                        t=t+1
                else:
                    data_c+=1

            else:
                #原始規則不再適用：網址失效/網頁改版
               #if error_count > 50:
                 #   web_id_error+= [web_id]
                 #   break
                error_count+= 1
                error_id[f] = {'web_id':web_id,'wrong_type':3,'title':title,'url':url,'title_rule':rule}
                f=f+1
        all_count=data_c+data_wrong_count+error_count+url_wrong_count+url_wrong_count
        
        data_count[j]={'web_id':web_id,
                       'data_correct':format(float(data_c)/float(all_count),'.2f'),
                       'data_wrong_count':format(float(data_wrong_count)/float(all_count),'.2f'),
                       'error_count':format(float(error_count)/float(all_count),'.2f'),
                       'url_wrong_count':format(float(url_wrong_count)/float(all_count),'.2f'), 
                       'no_rule':no_rule}
    
        j+=1
        if float(format(float(data_wrong_count)/float(all_count),'.2f'))>0.4:
            toemail(f"""title相似度<0.5:url相同,物品有變動""",f"""\nweb_id={web_id}\n\ndata_wrong_ratio={format(float(data_wrong_count)/float(all_count)*100,'.2f')}%\n\ntitle_rule={rule}""")
            save2xlsx(data_wrong_id,'data_wrong_id',web_id)
        if float(format(float(error_count)/float(all_count),'.2f'))>0.4:
            toemail(f"""爬蟲規則不再適用:網址失效/網頁改版""",f"""\nweb_id={web_id}\n\nerror_ratio={format(float(error_count)/float(all_count)*100,'.2f')}%\n\ntitle_rule={rule}""")
            save2xlsx(error_id,'error_id',web_id)
        if float(format(float(url_wrong_count)/float(all_count),'.2f'))>0.4:
            toemail(f"""資料庫的url與抓取的url不一樣:網頁跳轉/url有更新""",f"""\nweb_id={web_id}\n\nurl_wrong_ratio={format(float(url_wrong_count)/float(all_count)*100,'.2f')}%\n\ntitle_rule={rule}""")
            save2xlsx(url_wrong_id,'url_wrong_id',web_id)
            
    if norule:
        toemail(f"""資料庫無爬蟲規則""",norule)
    df_map = pd.DataFrame.from_dict(data_count)
    df_map=df_map.T.set_index('web_id')
    df_map.to_excel('title_report.xlsx')





            SELECT

            web_id,product_id,title, url 
            FROM
                item_list
            
            

            


  0%|          | 0/175 [00:00<?, ?it/s]

web_id =lucca


  1%|          | 1/175 [00:46<2:15:26, 46.71s/it]

web_id =openlife


...

web_id =greatshop


 99%|█████████▉| 174/175 [5:06:23<01:49, 109.07s/it]

Complete!
web_id =funselect


100%|██████████| 175/175 [5:09:41<00:00, 106.18s/it]


Complete!


In [ ]:
error_id_df = pd.DataFrame.from_dict(error_id,"index")
error_id_df.to_excel(f"""title_report/error/{error_id_df['web_id'][0]}_report.xlsx""",index=False)

In [4]:
soup

<html><body><p>0</p></body></html>

In [9]:
content

NameError: name 'content' is not defined

In [6]:
rule

'str(re.findall(r"\\t(.\\w.*).",str(soup.find_all(\'title\')[0].text)))[2:-2]'

In [19]:
if not content:
    print(1)

1


In [18]:
try:
    content = soup.find('meta', attrs=eval(str(rule)))
except:
    content=None

In [ ]:
            soup = fetch_soup(url)
            content = soup.find('meta', attrs=eval(str(rule)))

In [ ]:
content

In [ ]:
url='https://buy2.tw/product_detail.php?product_sn=121'

In [ ]:
soup = fetch_soup(url)
content = soup.find('meta', attrs=eval(str(rule)))

In [ ]:
rule.replace('"',)

In [ ]:
content

In [ ]:
soup

In [ ]:
def save2xlsx(data_wrong_id,path,web_id):
    data_wrong_id_df = pd.DataFrame.from_dict(data_wrong_id,"index")
    data_wrong_id_df.to_excel(f"""title_report/{path}/{web_id}_report.xlsx""",index=False)
    return

In [ ]:
soup

In [ ]:
rule[]

In [ ]:
web_id_c

In [ ]:
web_id_c=list(report[report['data_correct']<0.8].web_id)

In [ ]:
report=pd.read_excel('title_report.xlsx')
web_id_c=list(report[report['data_correct']>0.8].web_id)
web_id_d=[i for i in web_id_list if i not in web_id_c]

In [ ]:
web_id_c=list(report[report['data_correct']>0.8].web_id)

In [ ]:
import datetime

In [ ]:
timenow=datetime.datetime.now()

In [ ]:
str(timenow)[:19]

In [ ]:
            soup = fetch_soup(url)
            content = soup.find('meta', attrs=rule)

In [ ]:
timenow=datetime.datetime.now()
timenow=str(timenow)[:19]
with open(timenow,"w") as f:
    f.write("text")

In [ ]:
re.search(r'https...shop.kgcheck.com.tw.product_detail.php.product_sn=90',str(soup))

In [ ]:
soup = fetch_soup(url)

In [ ]:
https://shop.kgcheck.com.tw/product_detail?product_sn=90

In [ ]:
https://shop.kgcheck.com.tw/product_detail.php?product_sn=722

In [ ]:
soup

In [ ]:
'https://shop.kgcheck.com.tw/product_detail.php?product_sn=90'

In [ ]:
str(url_wrong_id).replace(']','\n')

In [ ]:
soup

In [ ]:
soup1=str(soup)

In [ ]:
url

In [ ]:
import re

In [ ]:
soup

In [ ]:
re.search(url,str(soup))

In [ ]:
if re.search(url,str(soup)):
    print(a)

In [ ]:
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import smtplib
def toemail(title,dict_):
    content = MIMEMultipart()  #建立MIMEMultipart物件
    content["subject"] = f"{title}"  #郵件標題
    content["from"] = "yu.jiang@avividai.com"  #寄件者
    content["to"] = "yu.jiang@avividai.com" #收件者
    content.attach(MIMEText(str(dict_).replace('}','\n')))  #郵件內容
    with smtplib.SMTP(host="smtp.gmail.com", port="587") as smtp:  # 設定SMTP伺服器
        try:
            smtp.ehlo()  # 驗證SMTP伺服器
            smtp.starttls()  # 建立加密傳輸
            smtp.login("yu.jiang@avividai.com", "mfitqgprhdpanvll")  # 登入寄件者gmail
            smtp.send_message(content)  # 寄送郵件
            print("Complete!")
        except Exception as e:
            print("Error message: ", e)
    return

In [ ]:
toemail('data count','@Yu Jiang')

In [ ]:
    query = f"""SELECT* FROM ecom_usertag limit 1000"""
    print(query)
    data = DBhelper('missioner').ExecuteSelect(query)
    df_article = pd.DataFrame(data, columns=['id', 'web_id', 'uuid', 'guid','usertag','product_id','date','add_time','update_time'])

In [ ]:
df_article.to_excel('ecom_usertag.xlsx',index=False)